<a href="https://colab.research.google.com/github/regenbuild/BIM2BEM/blob/main/TopologicalGraphDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
!pip install scipy
!pip install topologicpy

In [117]:
from scipy.spatial import ConvexHull, Delaunay, Voronoi, delaunay_plot_2d, convex_hull_plot_2d, voronoi_plot_2d, KDTree

import numpy as np
import pandas as pd

from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex

from topologicpy.Topology import Topology
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary

import csv
import re

In [118]:
from google.colab import files
uploaded = files.upload()


Saving SlabsWallsZones.csv to SlabsWallsZones (1).csv


In [120]:
import io
points_df = pd.read_csv(io.StringIO(uploaded['SlabsWallsZones (1).csv'].decode('utf-8')), names= ['id', 'name', 'x', 'y', 'z', 'indices'])


In [121]:
print(points_df.head())

                                 id  \
0  536801b326a11a0fabbd0d31cdedeb48   
1  536801b326a11a0fabbd0d31cdedeb48   
2  536801b326a11a0fabbd0d31cdedeb48   
3  536801b326a11a0fabbd0d31cdedeb48   
4  536801b326a11a0fabbd0d31cdedeb48   

                                                name      x      y     z  \
0  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29  10.39  5.41   
1  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29  10.39  0.49   
2  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.44  10.39  0.49   
3  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.44  10.39  5.41   
4  IfcWall/Basic Wall:WA_Ext_BrickWall_64-(2+1)_P... -46.29   0.09  5.41   

                                             indices  
0  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
1  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
2  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
3  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4, 0, 3, 3, 4, 3, ...  
4  [3, 0, 1, 2, 3, 0, 2, 3, 3, 4,

# Required Functions

In [122]:
def from_vertices_to_points_3D(vertices):
  return [[vertex.X(), vertex.Y(), vertex.Z()] for vertex in vertices]

def from_points_to_vertices_3D(points):
  return [Vertex.ByCoordinates(point[0], point[1], point[2]) for point in points]

In [123]:
def make_faces_from_indices(points, face_indices):
  l = len(face_indices)
  i = 0
  triangles = []

  while i < l:
    curr = face_indices[i]
    i+=1
    lis = face_indices[i:curr+i]
    triangles.append(lis)

    i+=curr

  faces_list = []

  for triangle in triangles:
    v1 = points[triangle[0]]
    v2 = points[triangle[1]]
    v3 = points[triangle[2]]
    v_list = from_points_to_vertices_3D([v1, v2, v3])
    f = Face.ByVertices(v_list)
    faces_list.append(f)

  return faces_list



In [124]:
WALL_REG = r"(.*IfcWall.*)"
SLAB_REG = r"(.*IfcSlab.*)"
WINDOW_REG = r"(.*IfcWindow.*)"
COLUMN_REG = r"(.*IfcColumn.*)"
DOOR_REG = r"(.*IfcDoor.*)"
SPACE_REG = r"(.*IfcSpace.*)"

# Topology Building

In [125]:
processed = {}

for tup in points_df.itertuples():
  # print(tup[2])
  if re.findall(SLAB_REG, tup[2]) or re.findall(SPACE_REG, tup[2]):
    # processed = {}
    if tup[1] not in processed:
      processed[tup[1]] = [[tup[2], tup[3], tup[4], tup[5], tup[6]]]
    else:
      processed[tup[1]].append([tup[2], tup[3], tup[4], tup[5], tup[6]])

packed = {}

for key, values in processed.items():
  coords = []

  for value in values:
    coords.append([value[1], value[2], value[3]])

  packed[key] = [values[0][0], coords, [int(i) for i in values[0][4][1:-1].split(", ")]]

after_packed = {}

for key, value in packed.items():
  if value[0] not in after_packed:
    after_packed[value[0]] = [[key, value[1], value[2]]]
  else:
    after_packed[value[0]].append([key, value[1], value[2]])

# for key, value in after_packed.items():
#   print(key, value)


ready_els = {}

for key, values in after_packed.items():
  cells = []
  for value in values:
    # print(value)
    c = Cell.ByFaces(make_faces_from_indices(points=value[1], face_indices=value[2]))
    cells.append(c)

  # print(len(cells))

  if len(cells) > 1:
    # print('complex el')
    cc = CellComplex.ByCells(cells)
    simp = CellComplex.ExternalBoundary(cc)
    ready_els[key] = simp
  else:
    # print('simple el')
    ready_els[key] = cells[0]

  # ready_els[key] = cells

In [126]:
Topology.Show(list(ready_els.values()), renderer='colab', vertexSize=4)

In [129]:
cc = CellComplex.ByCells(list(ready_els.values()))

In [131]:
g = Graph.ByTopology(cc)
Topology.Show(g, cc, renderer='colab', vertexSize=4)

Face.ByWire - Error: The input wire parameter is not a closed topologic wire. Returning None.
caller name: Triangulate
